# Autoencoder como detector de fracturas oseas
El autoencoder se entrena SOLO con imagenes normales 
(not fractured) y usa el error de reconstruccion como
medida de anomalia.


In [1]:
import os
import pandas as pd
from PIL import Image, ImageFile

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm import tqdm
import matplotlib.pyplot as plt

# permitir imagenes truncadas
ImageFile.LOAD_TRUNCATED_IMAGES = True

# reproducibilidad
torch.manual_seed(42)

# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cpu


In [2]:
# crear dataframes desde carpetas

def create_dataframe(root_path):
    filepaths = []
    labels = []

    if not os.path.exists(root_path):
        print(f"Error: La ruta {root_path} no existe.")
        return None

    for folder in os.listdir(root_path):
        folder_path = os.path.join(root_path, folder)

        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    fpath = os.path.join(folder_path, file)
                    filepaths.append(fpath)
                    labels.append(folder)

    df = pd.DataFrame({
        'filepath': filepaths,
        'label': labels
    })
    return df


train_dir = "data/Bone_Fracture_Binary_Classification/train"
val_dir   = "data/Bone_Fracture_Binary_Classification/val"
test_dir  = "data/Bone_Fracture_Binary_Classification/test"

train_df = create_dataframe(train_dir)
val_df   = create_dataframe(val_dir)
test_df  = create_dataframe(test_dir)

print("Train:", len(train_df))
print("Val:", len(val_df))
print("Test:", len(test_df))


Train: 9246
Val: 829
Test: 506


In [3]:
#limpieza de imagenes
def clean_dataframe(df):
    indices_to_drop = []
    print(f"Verificando {len(df)} imagenes...")

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        try:
            img = Image.open(row["filepath"])
            img.load()
        except:
            indices_to_drop.append(idx)

    print(f"Imagenes corruptas eliminadas: {len(indices_to_drop)}")
    return df.drop(indices_to_drop).reset_index(drop=True)


train_df = clean_dataframe(train_df)
val_df   = clean_dataframe(val_df)
test_df  = clean_dataframe(test_df)

print("Despues de limpieza:")
print("Train:", len(train_df))
print("Val:", len(val_df))
print("Test:", len(test_df))

Verificando 9246 imagenes...


100%|██████████| 9246/9246 [01:50<00:00, 83.84it/s] 


Imagenes corruptas eliminadas: 0
Verificando 829 imagenes...


100%|██████████| 829/829 [00:10<00:00, 78.43it/s] 


Imagenes corruptas eliminadas: 0
Verificando 506 imagenes...


100%|██████████| 506/506 [00:11<00:00, 45.86it/s]

Imagenes corruptas eliminadas: 0
Despues de limpieza:
Train: 9246
Val: 829
Test: 506


In [4]:
#dataset y dataloaders

class BoneFractureDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe.reset_index(drop=True)
        self.transform = transform
        
        # not fractured -> 0 (NORMAL)
        # fractured     -> 1 (ANOMALIA)
        self.label_map = {
            "not fractured": 0,
            "fractured": 1
        }

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['filepath']
        label_str = self.dataframe.iloc[idx]['label']
        
        image = Image.open(img_path).convert("L")
        label = self.label_map[label_str]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.float32)


transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])


train_dataset = BoneFractureDataset(train_df, transform=transform)
val_dataset   = BoneFractureDataset(val_df, transform=transform)
test_dataset  = BoneFractureDataset(test_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("Train batches:", len(train_loader))
print("Val batches:", len(val_loader))
print("Test batches:", len(test_loader))

Train batches: 289
Val batches: 26
Test batches: 16


In [5]:
#filtranos solo clase normal
normal_indices_train = [
    idx for idx in range(len(train_dataset))
    if train_dataset.dataframe.iloc[idx]['label'] == "not fractured"
]

train_normal_dataset = torch.utils.data.Subset(train_dataset, normal_indices_train)
train_normal_loader  = DataLoader(train_normal_dataset, batch_size=32, shuffle=True)

print("Imagenes SOLO NORMALES:", len(train_normal_dataset))

Imagenes SOLO NORMALES: 4640


In [6]:
#autoencoder 
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, 2, 1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, 2, 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, 2, 1),
            nn.ReLU()
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 3, 2, 1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, 2, 1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, 2, 1, output_padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))


ae_model = ConvAutoencoder().to(device)
criterion_ae = nn.MSELoss()
optimizer_ae = torch.optim.Adam(ae_model.parameters(), lr=1e-3)

print(ae_model)


ConvAutoencoder(
  (encoder): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): ReLU()
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(64, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(32, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (3): ReLU()
    (4): ConvTranspose2d(16, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (5): Tanh()
  )
)


In [7]:
#entrenamiento solo con imagenes normales
num_epochs_ae = 25
train_losses_ae = []

for epoch in range(num_epochs_ae):
    ae_model.train()
    running_loss = 0.0

    for images, _ in train_normal_loader:
        images = images.to(device)

        outputs = ae_model(images)
        loss = criterion_ae(outputs, images)

        optimizer_ae.zero_grad()
        loss.backward()
        optimizer_ae.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_normal_loader)
    train_losses_ae.append(avg_loss)

    print(f"Epoch [{epoch+1}/{num_epochs_ae}] | AE Train Recon Loss: {avg_loss:.6f}")

Epoch [1/25] | AE Train Recon Loss: 0.116365
Epoch [2/25] | AE Train Recon Loss: 0.016336
Epoch [3/25] | AE Train Recon Loss: 0.012124
Epoch [4/25] | AE Train Recon Loss: 0.009908
Epoch [5/25] | AE Train Recon Loss: 0.008244
Epoch [6/25] | AE Train Recon Loss: 0.006942
Epoch [7/25] | AE Train Recon Loss: 0.006058
Epoch [8/25] | AE Train Recon Loss: 0.005492
Epoch [9/25] | AE Train Recon Loss: 0.005060
Epoch [10/25] | AE Train Recon Loss: 0.004724
Epoch [11/25] | AE Train Recon Loss: 0.004426
Epoch [12/25] | AE Train Recon Loss: 0.004208
Epoch [13/25] | AE Train Recon Loss: 0.003977
Epoch [14/25] | AE Train Recon Loss: 0.003833
Epoch [15/25] | AE Train Recon Loss: 0.003615
Epoch [16/25] | AE Train Recon Loss: 0.003484
Epoch [17/25] | AE Train Recon Loss: 0.003354
Epoch [18/25] | AE Train Recon Loss: 0.003211
Epoch [19/25] | AE Train Recon Loss: 0.003101
Epoch [20/25] | AE Train Recon Loss: 0.002985
Epoch [21/25] | AE Train Recon Loss: 0.002880
Epoch [22/25] | AE Train Recon Loss: 0.0028

In [8]:
# ============================================================
# CHUNK FINAL - AUTOENCODER COMO CLASIFICADOR (ANOMALY DETECTOR)
# ============================================================

def reconstruction_errors(ae_model, data_loader, device):
    ae_model.eval()
    all_errors = []
    all_labels = []

    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)

            recon = ae_model(images)

            # MSE por pixel
            loss_per_pixel = F.mse_loss(recon, images, reduction="none")
            # promedio por imagen
            loss_per_image = loss_per_pixel.view(loss_per_pixel.size(0), -1).mean(dim=1)

            all_errors.append(loss_per_image.cpu())
            all_labels.append(labels.cpu())

    all_errors = torch.cat(all_errors)
    all_labels = torch.cat(all_labels)
    return all_errors, all_labels


# ------------------------------------------------------------
# 1. CALCULAR ERRORES EN VALIDACION
# ------------------------------------------------------------

val_errors, val_labels = reconstruction_errors(ae_model, val_loader, device)

# SOLO errores de la clase NORMAL (0 = not fractured)
normal_val_errors = val_errors[val_labels == 0]

# UMBRAL = media + 2 * desviacion estandar
threshold = normal_val_errors.mean() + 2 * normal_val_errors.std()

print("Umbral de anomalía (recon error):", threshold.item())


# ------------------------------------------------------------
# 2. FUNCION PARA CALCULAR ACCURACY CON UMBRAL
# ------------------------------------------------------------

def evaluate_ae_classifier(ae_model, data_loader, device, threshold):
    ae_model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)

            recon = ae_model(images)

            loss_per_pixel = F.mse_loss(recon, images, reduction="none")
            loss_per_image = loss_per_pixel.view(loss_per_pixel.size(0), -1).mean(dim=1)

            # prediccion: 0 si <= umbral, 1 si > umbral
            preds = (loss_per_image > threshold.to(device)).float()

            total += labels.size(0)
            correct += (preds == labels).sum().item()

    acc = 100.0 * correct / total
    return acc


# ------------------------------------------------------------
# 3. ACCURACY FINAL DEL AUTOENCODER
# ------------------------------------------------------------

ae_train_acc = evaluate_ae_classifier(ae_model, train_loader, device, threshold)
ae_val_acc   = evaluate_ae_classifier(ae_model, val_loader, device, threshold)
ae_test_acc  = evaluate_ae_classifier(ae_model, test_loader, device, threshold)

print("\n=== RESULTADOS FINALES DEL AUTOENCODER ===")
print(f"AE Accuracy Train: {ae_train_acc:.2f}%")
print(f"AE Accuracy Val:   {ae_val_acc:.2f}%")
print(f"AE Accuracy Test:  {ae_test_acc:.2f}%")


c:\Users\diana\anaconda3\Lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Umbral de anomalía (recon error): 0.004583260044455528

=== RESULTADOS FINALES DEL AUTOENCODER ===
AE Accuracy Train: 59.03%
AE Accuracy Val:   68.15%
AE Accuracy Test:  55.53%


El autoencoder convolucional fue entrenado exclusivamente con imágenes de la clase not fractured como clase normal. Posteriormente, se utilizó el error de reconstrucción para definir un umbral de anomalía, permitiendo clasificar nuevas imágenes como normales o anómalas. El desempeño se evaluó mediante accuracy en los conjuntos de entrenamiento, validación y prueba.